In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, year, to_date, broadcast, lit, split, when, max,min, to_timestamp, avg
import findspark

findspark.init()

# create a SparkSession
spark = SparkSession.builder.appName("PlayStoreAppData").getOrCreate()


In [18]:
# load a CSV file into a DataFrame
df = spark.read.csv("Data/playstore.csv",  header=True)


In [19]:
#Checking If bucketing is enabled or not 
spark.conf.get("spark.sql.sources.bucketing.enabled")

'true'

In [20]:
#Show list of columns name
print(df.columns)

['_c0', 'appId', 'developer', 'developerId', 'developerWebsite', 'free', 'genre', 'genreId', 'inAppProductPrice', 'minInstalls', 'offersIAP', 'originalPrice', 'price', 'ratings', 'len screenshots', 'adSupported', 'containsAds', 'reviews', 'releasedDayYear', 'sale', 'score', 'summary', 'title', 'updated', 'histogram1', 'histogram2', 'histogram3', 'histogram4', 'histogram5', 'releasedDay', 'releasedYear', 'releasedMonth', 'dateUpdated', 'minprice', 'maxprice', 'ParseReleasedDayYear']


In [21]:
#Rename the index column
df = df.withColumnRenamed("_c0", "Index")
print(df.columns)

['Index', 'appId', 'developer', 'developerId', 'developerWebsite', 'free', 'genre', 'genreId', 'inAppProductPrice', 'minInstalls', 'offersIAP', 'originalPrice', 'price', 'ratings', 'len screenshots', 'adSupported', 'containsAds', 'reviews', 'releasedDayYear', 'sale', 'score', 'summary', 'title', 'updated', 'histogram1', 'histogram2', 'histogram3', 'histogram4', 'histogram5', 'releasedDay', 'releasedYear', 'releasedMonth', 'dateUpdated', 'minprice', 'maxprice', 'ParseReleasedDayYear']


In [24]:
#Picking the important columns for the operations
columns = ['Index', 'appId', 'developer', 'developerId', 'developerWebsite', 'free', 'genreId', 'inAppProductPrice',
       'minInstalls', 'originalPrice', 'price', 'ratings', 'adSupported', 'containsAds', 'reviews',
       'releasedDayYear', 'sale', 'score', 'title', 'releasedDay', 'releasedYear', 'releasedMonth', 'dateUpdated',
       'minprice', 'maxprice']

df =  df.select(columns)

In [25]:
df.printSchema()

root
 |-- Index: string (nullable = true)
 |-- appId: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- developerId: string (nullable = true)
 |-- developerWebsite: string (nullable = true)
 |-- free: string (nullable = true)
 |-- genreId: string (nullable = true)
 |-- inAppProductPrice: string (nullable = true)
 |-- minInstalls: string (nullable = true)
 |-- originalPrice: string (nullable = true)
 |-- price: string (nullable = true)
 |-- ratings: string (nullable = true)
 |-- adSupported: string (nullable = true)
 |-- containsAds: string (nullable = true)
 |-- reviews: string (nullable = true)
 |-- releasedDayYear: string (nullable = true)
 |-- sale: string (nullable = true)
 |-- score: string (nullable = true)
 |-- title: string (nullable = true)
 |-- releasedDay: string (nullable = true)
 |-- releasedYear: string (nullable = true)
 |-- releasedMonth: string (nullable = true)
 |-- dateUpdated: string (nullable = true)
 |-- minprice: string (nullable = true)
 |--

    The above cell output shows that fields like price, rating, minprice, maxprice, minInstalls etc
    are string we need to convert them to there required datatype like price, score,rating,minprice 
    etc should be converted to float or int, ParseReleasedDayYear and dateUpdated should be converted 
    to timestamp

In [27]:
# Casting column datatype to sutiable datatype
#Float
df = df.withColumn("price", col("price").cast("float"))
df = df.withColumn("score", col("score").cast("float"))
df = df.withColumn("minprice", col("minprice").cast("float"))
df = df.withColumn("maxprice", col("maxprice").cast("float"))
#Int
df = df.withColumn("free", col("free").cast("int"))
df = df.withColumn("ratings", col("ratings").cast("int"))
df = df.withColumn("minInstalls", col("minInstalls").cast("int"))
df = df.withColumn("adSupported", col("adSupported").cast("int"))
df = df.withColumn("containsAds", col("containsAds").cast("int"))
df = df.withColumn("reviews", col("reviews").cast("int"))
df = df.withColumn("releasedYear", col("releasedYear").cast("int"))


# Parse ReleasedDayYear column to timestamp and date (assuming format "yyyy-MM-dd")
df = df.withColumn("dateUpdated", to_timestamp(col("dateUpdated")))

In [28]:
#Print Schema to validate chages
df.printSchema()

root
 |-- Index: string (nullable = true)
 |-- appId: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- developerId: string (nullable = true)
 |-- developerWebsite: string (nullable = true)
 |-- free: integer (nullable = true)
 |-- genreId: string (nullable = true)
 |-- inAppProductPrice: string (nullable = true)
 |-- minInstalls: integer (nullable = true)
 |-- originalPrice: string (nullable = true)
 |-- price: float (nullable = true)
 |-- ratings: integer (nullable = true)
 |-- adSupported: integer (nullable = true)
 |-- containsAds: integer (nullable = true)
 |-- reviews: integer (nullable = true)
 |-- releasedDayYear: string (nullable = true)
 |-- sale: string (nullable = true)
 |-- score: float (nullable = true)
 |-- title: string (nullable = true)
 |-- releasedDay: string (nullable = true)
 |-- releasedYear: integer (nullable = true)
 |-- releasedMonth: string (nullable = true)
 |-- dateUpdated: timestamp (nullable = true)
 |-- minprice: float (nullable = tru

Creating Buckets based on columns in the data

In [36]:
df.write.bucketBy(12, 'releasedYear').saveAsTable('releasedYear_bucket', format='csv')

In [44]:
df.write.bucketBy(10, 'developerId').saveAsTable('developer_bucket', format='csv')

In [43]:
df.write.bucketBy(10, 'reviews').saveAsTable('reviews_bucket', format='csv')

In [48]:
df.write.bucketBy(10, 'price').saveAsTable('price_bucket', format='csv')

In [49]:
df.write.bucketBy(10, 'genreId').saveAsTable('genre_bucket', format='csv')

In [50]:
df.write.bucketBy(12, 'minInstalls').saveAsTable('Installation_bucket', format='csv')

In [57]:
#Operations on price bucket
price = spark.table("price_bucket")
price_df = price.limit(5).toPandas()
price_df

,Index,appId,developer,developerId,developerWebsite,free,genreId,inAppProductPrice,minInstalls,originalPrice,...,releasedDayYear,sale,score,title,releasedDay,releasedYear,releasedMonth,dateUpdated,minprice,maxprice
0,899795,app.sangreazul,3Dent medical s. r. o.,5142856662213353785,http://www.3Dent.sk,1,MEDICAL,None,100,None,...,"Jun 4, 2018",0,0.0,StrojCHECK by SangreAzul - Invisalign with A.I.,4,2018,Jun,2022-01-19 11:32:05,NaN,0.0
1,1000944,com.unitedstatesradios.radioquebec,Multiple Radios Online AM FM Apps,6263290699348208647,https://radiosamfmgratis.blogspot.com,1,MUSIC_AND_AUDIO,None,1,None,...,"Oct 31, 2020",0,0.0,US Radio Quebec App Radio List,31,2020,Oct,2020-10-31 19:15:07,NaN,0.0
2,899796,com.MobileApp.CareIs,CAREis,CAREis,http://www.careis.net,1,MEDICAL,None,100,None,...,"Jun 5, 2018",0,0.0,CAREis,5,2018,Jun,2022-06-21 14:49:55,NaN,0.0
3,1000945,com.audiostreamvolt.ecochiquiano,AudioStreamVolt.com,5057669035764163988,https://www.streamingeconomico.com/,1,MUSIC_AND_AUDIO,None,5,None,...,"Nov 29, 2020",0,0.0,EcoChiquiano,29,2020,Nov,2020-12-01 16:41:33,NaN,0.0
4,899797,com.sky.injurytrak,InjuryTrak Software,InjuryTrak+Software,http://injurytrak.com,1,MEDICAL,None,50,None,...,"Jun 5, 2018",0,0.0,InjuryTrak,5,2018,Jun,2020-08-25 16:21:43,NaN,0.0


In [72]:
#1) Finding the top 10 costliest app in the data and developer developed it and genre it belong
price.createOrReplaceTempView('price_table')
app_cost_result = spark.sql("""SELECT appId, developerId, genreId, price
                            FROM price_table
                            ORDER BY price desc
                            LIMIT 10
                            """)
app_cost_result.show()
app_cost_result.write.csv("output/price_bucket/app_cost_result.csv", header=True)

+--------------------+--------------------+-------------+-----------+
|               appId|         developerId|      genreId|      price|
+--------------------+--------------------+-------------+-----------+
|     명언여행 - 명언|                명상|       멘토글|1.5616224E9|
|PT. Smartfren Tel...|mobapp@smartfren.com|         True|    77278.0|
|          M2Catalyst|support@m2catalys...|         True|     9938.0|
|        Digitalchemy|FractionCalculato...|        Tools|     8007.0|
| Internet Speed Test|support@speedspot...|        Tools|     6898.0|
|Comcast Cable Cor...|xfinity_tvapp@com...|         True|     4141.0|
|             NETGEAR|netgearteam@netge...|         True|     3928.0|
|         Golden Frog|support@goldenfro...|         True|     2717.0|
|com.altshift.oren...| 5549044713487227941|GAME_STRATEGY|     1100.0|
|         PeopleReady|appsupport@truebl...|         True|      917.0|
+--------------------+--------------------+-------------+-----------+



In [85]:
#2) Costilest app across each genre
top_costliest_app_genre =spark.sql( """
    SELECT appId,developer,genreId,maxprice
    FROM (
        SELECT
            appId, developer, genreId, maxprice,
            ROW_NUMBER() OVER (PARTITION BY genreId ORDER BY maxprice DESC) AS rank
        FROM
            price_table
    ) ranked
    WHERE rank = 1
    ORDER BY maxprice desc
""")
top_costliest_app_genre.show()
top_costliest_app_genre.write.csv("output/price_bucket/top_costliest_app_genre.csv", header=True)

+--------------------+----------------------------+-------------------+------------+
|               appId|                   developer|            genreId|    maxprice|
+--------------------+----------------------------+-------------------+------------+
|成都河兴科技有限公司|        hexingkejiyzx@gma...|           SHOPPING|1.66012902E9|
|        Macro School|        macrotipsbet24@gm...|             SPORTS|1.65939392E9|
|Northern Unicorn ...|        ineedthisapp10@gm...|      COMMUNICATION|1.65787251E9|
|    Apps for Anybody|        contact.topapps@g...|           BUSINESS|1.65725478E9|
|         Tomlibo.com|         service@tomlibo.com|             SOCIAL| 1.6567872E9|
|            Fat Cats|                         LLC|               True|1.65423846E9|
|   Frihed Mobile LTD|        sam.cheng@frihed....|            MEDICAL|1.65154227E9|
|      learn language|         daohuynh7@gmail.com|          EDUCATION|1.64822272E9|
|              Veysel|        bilkorkmaz1983@gm...|              TOOLS|1.64

In [75]:
#Operations on Installation_bucket bucket
Install = spark.table("Installation_bucket")
Install_df = price.limit(5).toPandas()
Install_df

,Index,appId,developer,developerId,developerWebsite,free,genreId,inAppProductPrice,minInstalls,originalPrice,...,releasedDayYear,sale,score,title,releasedDay,releasedYear,releasedMonth,dateUpdated,minprice,maxprice
0,899795,app.sangreazul,3Dent medical s. r. o.,5142856662213353785,http://www.3Dent.sk,1,MEDICAL,None,100,None,...,"Jun 4, 2018",0,0.0,StrojCHECK by SangreAzul - Invisalign with A.I.,4,2018,Jun,2022-01-19 11:32:05,NaN,0.0
1,1000944,com.unitedstatesradios.radioquebec,Multiple Radios Online AM FM Apps,6263290699348208647,https://radiosamfmgratis.blogspot.com,1,MUSIC_AND_AUDIO,None,1,None,...,"Oct 31, 2020",0,0.0,US Radio Quebec App Radio List,31,2020,Oct,2020-10-31 19:15:07,NaN,0.0
2,899796,com.MobileApp.CareIs,CAREis,CAREis,http://www.careis.net,1,MEDICAL,None,100,None,...,"Jun 5, 2018",0,0.0,CAREis,5,2018,Jun,2022-06-21 14:49:55,NaN,0.0
3,1000945,com.audiostreamvolt.ecochiquiano,AudioStreamVolt.com,5057669035764163988,https://www.streamingeconomico.com/,1,MUSIC_AND_AUDIO,None,5,None,...,"Nov 29, 2020",0,0.0,EcoChiquiano,29,2020,Nov,2020-12-01 16:41:33,NaN,0.0
4,899797,com.sky.injurytrak,InjuryTrak Software,InjuryTrak+Software,http://injurytrak.com,1,MEDICAL,None,50,None,...,"Jun 5, 2018",0,0.0,InjuryTrak,5,2018,Jun,2020-08-25 16:21:43,NaN,0.0


In [89]:
#3) top app across each genre in number of installation 
Install.createOrReplaceTempView('Installation_table')
top_apps_install = spark.sql( """
    SELECT appId, developerId, genreId, minInstalls
    FROM (
        SELECT appId, developerId, genreId, minInstalls,
            ROW_NUMBER() OVER (PARTITION BY genreId ORDER BY minInstalls DESC) AS rank
        FROM
            Installation_table
    ) ranked
    WHERE rank <= 1
    ORDER BY minInstalls desc
""")
top_apps_install.show()
top_apps_install.write.csv("output/Installation_bucket/top_apps_install.csv", header=True)

+--------------------+--------------------+-------------------+-----------+
|               appId|         developerId|            genreId|minInstalls|
+--------------------+--------------------+-------------------+-----------+
|com.kiloo.subwaysurf| 7363891306616760846|        GAME_ARCADE| 1000000000|
|   com.spotify.music|          Spotify+AB|    MUSIC_AND_AUDIO| 1000000000|
|  com.dts.freefireth|Garena+Internatio...|        GAME_ACTION| 1000000000|
|    com.skype.raider|               Skype|      COMMUNICATION| 1000000000|
|com.netflix.media...|       Netflix,+Inc.|      ENTERTAINMENT| 1000000000|
|com.google.androi...| 5700313618786177705|   TRAVEL_AND_LOCAL| 1000000000|
|com.google.androi...| 5700313618786177705| NEWS_AND_MAGAZINES| 1000000000|
|com.sec.android.a...| 5200379633052405703|    PERSONALIZATION| 1000000000|
|com.google.androi...| 5700313618786177705|BOOKS_AND_REFERENCE| 1000000000|
|com.samsung.knox....| 5200379633052405703|           BUSINESS| 1000000000|
|com.mxtech.

In [91]:
#Operations on review bucket
review = spark.table("reviews_bucket")
review_df = price.limit(5).toPandas()
review_df

,Index,appId,developer,developerId,developerWebsite,free,genreId,inAppProductPrice,minInstalls,originalPrice,...,releasedDayYear,sale,score,title,releasedDay,releasedYear,releasedMonth,dateUpdated,minprice,maxprice
0,899795,app.sangreazul,3Dent medical s. r. o.,5142856662213353785,http://www.3Dent.sk,1,MEDICAL,None,100,None,...,"Jun 4, 2018",0,0.0,StrojCHECK by SangreAzul - Invisalign with A.I.,4,2018,Jun,2022-01-19 11:32:05,NaN,0.0
1,1000944,com.unitedstatesradios.radioquebec,Multiple Radios Online AM FM Apps,6263290699348208647,https://radiosamfmgratis.blogspot.com,1,MUSIC_AND_AUDIO,None,1,None,...,"Oct 31, 2020",0,0.0,US Radio Quebec App Radio List,31,2020,Oct,2020-10-31 19:15:07,NaN,0.0
2,899796,com.MobileApp.CareIs,CAREis,CAREis,http://www.careis.net,1,MEDICAL,None,100,None,...,"Jun 5, 2018",0,0.0,CAREis,5,2018,Jun,2022-06-21 14:49:55,NaN,0.0
3,1000945,com.audiostreamvolt.ecochiquiano,AudioStreamVolt.com,5057669035764163988,https://www.streamingeconomico.com/,1,MUSIC_AND_AUDIO,None,5,None,...,"Nov 29, 2020",0,0.0,EcoChiquiano,29,2020,Nov,2020-12-01 16:41:33,NaN,0.0
4,899797,com.sky.injurytrak,InjuryTrak Software,InjuryTrak+Software,http://injurytrak.com,1,MEDICAL,None,50,None,...,"Jun 5, 2018",0,0.0,InjuryTrak,5,2018,Jun,2020-08-25 16:21:43,NaN,0.0


In [97]:
#4) top review free app accross each genreId
review.createOrReplaceTempView('review_table')
top_reviewed_free_apps = spark.sql("""
    SELECT appId,developerId,genreId,title,reviews
    FROM (
        SELECT appId, developerId, genreId, title, reviews,
            ROW_NUMBER() OVER (PARTITION BY genreId ORDER BY reviews DESC) AS rank
        FROM
            review_table
        WHERE
            free = 1
    ) ranked
    WHERE rank = 1
    ORDER BY reviews desc
""")
top_reviewed_free_apps.show()

top_reviewed_free_apps.write.csv("output/review_bucket/top_reviewed_free_apps.csv", header=True)

+--------------------+--------------------+-------------------+--------------------+-------+
|               appId|         developerId|            genreId|               title|reviews|
+--------------------+--------------------+-------------------+--------------------+-------+
|com.instagram.and...|           Instagram|             SOCIAL|           Instagram|4394018|
|   com.roblox.client| 5360036014478858866|     GAME_ADVENTURE|              Roblox|3179708|
|com.google.androi...| 5700313618786177705|      VIDEO_PLAYERS|             YouTube|2770308|
|   com.facebook.orca|Meta+Platforms,+Inc.|      COMMUNICATION|           Messenger|2370886|
|com.king.candycru...| 6577204690045492686|        GAME_CASUAL|    Candy Crush Saga|2017101|
|com.supercell.cla...| 6715068722362591614|      GAME_STRATEGY|      Clash of Clans|1949249|
|com.kiloo.subwaysurf| 7363891306616760846|        GAME_ARCADE|      Subway Surfers|1790042|
|   com.spotify.music|          Spotify+AB|    MUSIC_AND_AUDIO|Spotify

In [98]:
#5) top review paid app accross each genreId
top_reviewed_paid_apps = spark.sql("""
    SELECT appId,developerId,genreId,title,reviews
    FROM (
        SELECT appId, developerId, genreId, title, reviews,
            ROW_NUMBER() OVER (PARTITION BY genreId ORDER BY reviews DESC) AS rank
        FROM
            review_table
        WHERE
            free = 0
    ) ranked
    WHERE rank = 1
    ORDER BY reviews desc
""")
top_reviewed_free_apps.show()

top_reviewed_paid_apps.write.csv("output/review_bucket/top_reviewed_paid_apps.csv", header=True)

+--------------------+--------------------+-------------------+--------------------+-------+
|               appId|         developerId|            genreId|               title|reviews|
+--------------------+--------------------+-------------------+--------------------+-------+
|com.instagram.and...|           Instagram|             SOCIAL|           Instagram|4394018|
|   com.roblox.client| 5360036014478858866|     GAME_ADVENTURE|              Roblox|3179708|
|com.google.androi...| 5700313618786177705|      VIDEO_PLAYERS|             YouTube|2770308|
|   com.facebook.orca|Meta+Platforms,+Inc.|      COMMUNICATION|           Messenger|2370886|
|com.king.candycru...| 6577204690045492686|        GAME_CASUAL|    Candy Crush Saga|2017101|
|com.supercell.cla...| 6715068722362591614|      GAME_STRATEGY|      Clash of Clans|1949249|
|com.kiloo.subwaysurf| 7363891306616760846|        GAME_ARCADE|      Subway Surfers|1790042|
|   com.spotify.music|          Spotify+AB|    MUSIC_AND_AUDIO|Spotify

In [99]:
#Operations on developer bucket
developers = spark.table("developer_bucket")
developers_df = developers.limit(5).toPandas()
developers_df

,Index,appId,developer,developerId,developerWebsite,free,genreId,inAppProductPrice,minInstalls,originalPrice,...,releasedDayYear,sale,score,title,releasedDay,releasedYear,releasedMonth,dateUpdated,minprice,maxprice
0,2388094,com.ssaurel.pedometer.pro,Sylvain Saurel,6924401024188312025,http://www.ssaurel.com,0,HEALTH_AND_FITNESS,None,50,None,...,"Aug 14, 2016",0,0.0,Pedometer Step Counter Pro,14,2016,Aug,2016-08-14 07:27:57,NaN,0.00
1,2489820,com.rk141.hindijokes,Rahul Katiyar,Rahul+Katiyar,https://x4195ae96.app-ads-txt.com,1,ENTERTAINMENT,None,50,None,...,"Sep 29, 2021",0,0.0,Hindi Funny Jokes,29,2021,Sep,2021-09-29 09:46:54,NaN,0.00
2,2388096,com.zdn35.music.songs.sleepingsounds,Limoni Audio Sounds,Limoni+Audio+Sounds,https://audiolimon2021.000webhostapp.com/,1,HEALTH_AND_FITNESS,$1.99 - $5.99 per item,1000,None,...,"Jul 3, 2016",0,0.0,Noise sounds for sleeping. Whi,3,2016,Jul,2022-08-02 14:49:13,NaN,5.99
3,2489824,com.spiderapps.frases.amor,Spider apps,Spider+apps,https://appsandroid.xyz,1,ENTERTAINMENT,None,50,None,...,"Dec 9, 2021",0,0.0,Imágenes con frases amor 2022,9,2021,Dec,2021-12-09 19:22:19,NaN,0.00
4,2388102,jp.co.venturebank.lava.meditation,ホットヨガスタジオLAVA,%E3%83%9B%E3%83%83%E3%83%88%E3%83%A8%E3%82%AC%...,None,1,HEALTH_AND_FITNESS,$1.49 - $20.99 per item,50000,None,...,"Jun 30, 2016",0,0.0,MEISOON:LAVA監修・瞑想・マインドフルネス・安眠,30,2016,Jun,2022-08-18 03:21:58,NaN,20.99


In [103]:
developers.createOrReplaceTempView('developers_table')
#6) Count of developers with the most apps
count_most_apps_by_developer =spark.sql("""
    SELECT developerId, COUNT(*) AS app_count 
    FROM developers_table
    GROUP BY developerId
    ORDER BY app_count desc
    limit 10
""")
count_most_apps_by_developer.show()
count_most_apps_by_developer.write.csv("output/developer_bucket/count_most_apps_by_developer.csv", header=True)

+-------------------+---------+
|        developerId|app_count|
+-------------------+---------+
|      Subsplash+Inc|     4996|
|            ChowNow|     4949|
|            Phorest|     3754|
|8453266419614197800|     3210|
|          OrderYOYO|     3150|
| Apptegy+Play+Store|     2851|
|           Aradsoft|     2786|
|              COGUL|     2386|
|6950510449232387039|     2121|
|       Dantebus.com|     1861|
+-------------------+---------+



In [107]:
# 7) Write SQL query to find the developers with the most minInstalls
developers_with_most_mininstalls_query = spark.sql("""
    SELECT developerId, SUM(minInstalls) AS total_minInstalls
    FROM developers_table
    GROUP BY developerId
    ORDER BY total_minInstalls DESC
    LIMIT 10
""")
developers_with_most_mininstalls_query.show()
developers_with_most_mininstalls_query.write.csv("output/developer_bucket/developers_with_most_mininstalls_query.csv", header=True)

+--------------------+-----------------+
|         developerId|total_minInstalls|
+--------------------+-----------------+
| 5700313618786177705|      31031725329|
| 5200379633052405703|      15864892640|
| 6720847872553662727|       6440743734|
| 5113340212256272297|       5371015000|
| 5630538819012062144|       4361810000|
|Motorola+Mobility...|       2964667500|
|Huawei+Internet+S...|       2700000000|
|              VOODOO|       2248012700|
|    TikTok+Pte.+Ltd.|       2162000000|
|              ANT%2B|       2001000000|
+--------------------+-----------------+



In [109]:
#8)  developers with the most minInstalls in each genre
developers_with_most_mininstalls_genrewise =spark.sql("""
    SELECT developerId, genreId, total_minInstalls
    FROM (
        SELECT
            developerId,
            genreId,
            SUM(minInstalls) AS total_minInstalls,
            ROW_NUMBER() OVER (PARTITION BY genreId ORDER BY SUM(minInstalls) DESC) AS rn
        FROM
            developers_table
        GROUP BY
            developerId, genreId
    ) ranked
    WHERE rn = 1
    ORDER BY total_minInstalls desc
""")
developers_with_most_mininstalls_genrewise.show()
developers_with_most_mininstalls_genrewise.write.csv("output/developer_bucket/developers_with_most_mininstalls_genrewise.csv", header=True)

+--------------------+-------------------+-----------------+
|         developerId|            genreId|total_minInstalls|
+--------------------+-------------------+-----------------+
| 5700313618786177705|              TOOLS|       9658679327|
| 5700313618786177705|       PRODUCTIVITY|       6536210000|
| 5700313618786177705|      COMMUNICATION|       3630110000|
| 5630538819012062144|        GAME_CASUAL|       2900000000|
| 5700313618786177705|    MUSIC_AND_AUDIO|       2120000000|
| 5700313618786177705|   TRAVEL_AND_LOCAL|       2100000000|
| 4946022439885210717|   GAME_EDUCATIONAL|       1639600000|
|    TikTok+Pte.+Ltd.|             SOCIAL|       1511000000|
| 7019463006329470284| NEWS_AND_MAGAZINES|       1500000000|
|       Netflix,+Inc.|      ENTERTAINMENT|       1101120000|
|Garena+Internatio...|        GAME_ACTION|       1100000000|
|MX+Media+(formerl...|      VIDEO_PLAYERS|       1050000000|
|              VOODOO|        GAME_ARCADE|       1049806000|
| 5700313618786177705|  

In [111]:
#Operations on relesed year bucket
releasedYear = spark.table("releasedYear_bucket")
releasedYear_df = price.limit(5).toPandas()
releasedYear_df

,Index,appId,developer,developerId,developerWebsite,free,genreId,inAppProductPrice,minInstalls,originalPrice,...,releasedDayYear,sale,score,title,releasedDay,releasedYear,releasedMonth,dateUpdated,minprice,maxprice
0,899795,app.sangreazul,3Dent medical s. r. o.,5142856662213353785,http://www.3Dent.sk,1,MEDICAL,None,100,None,...,"Jun 4, 2018",0,0.0,StrojCHECK by SangreAzul - Invisalign with A.I.,4,2018,Jun,2022-01-19 11:32:05,NaN,0.0
1,1000944,com.unitedstatesradios.radioquebec,Multiple Radios Online AM FM Apps,6263290699348208647,https://radiosamfmgratis.blogspot.com,1,MUSIC_AND_AUDIO,None,1,None,...,"Oct 31, 2020",0,0.0,US Radio Quebec App Radio List,31,2020,Oct,2020-10-31 19:15:07,NaN,0.0
2,899796,com.MobileApp.CareIs,CAREis,CAREis,http://www.careis.net,1,MEDICAL,None,100,None,...,"Jun 5, 2018",0,0.0,CAREis,5,2018,Jun,2022-06-21 14:49:55,NaN,0.0
3,1000945,com.audiostreamvolt.ecochiquiano,AudioStreamVolt.com,5057669035764163988,https://www.streamingeconomico.com/,1,MUSIC_AND_AUDIO,None,5,None,...,"Nov 29, 2020",0,0.0,EcoChiquiano,29,2020,Nov,2020-12-01 16:41:33,NaN,0.0
4,899797,com.sky.injurytrak,InjuryTrak Software,InjuryTrak+Software,http://injurytrak.com,1,MEDICAL,None,50,None,...,"Jun 5, 2018",0,0.0,InjuryTrak,5,2018,Jun,2020-08-25 16:21:43,NaN,0.0


In [114]:
#9) Top 10 Years with the most released app 
releasedYear.createOrReplaceTempView('releasedYear_table')
most_realsed_year =spark.sql("""
    SELECT releasedYear, COUNT(*) as total_release from releasedYear_table
    GROUP BY releasedYear
    ORDER BY total_release desc
    limit 10
""")
most_realsed_year.show()
most_realsed_year.write.csv("output/releasedYear_bucket/most_realsed_year.csv", header=True)

+------------+-------------+
|releasedYear|total_release|
+------------+-------------+
|        2021|       716182|
|        2020|       696791|
|        2022|       586160|
|        2019|       514839|
|        2018|       298600|
|        2017|       218256|
|        2016|       145384|
|        2015|       104936|
|        2014|        72401|
|        2013|        44732|
+------------+-------------+



In [177]:
#10) App relased in each month of years and there rank
app_year_month = spark.sql("""
         Select *, DENSE_RANK() OVER(PARTITION BY releasedYear ORDER BY month_count DESC) as rank
         FROM(
             SELECT releasedYear, releasedMonth, COUNT(releasedMonth) as month_count  FROM releasedYear_table
             WHERE releasedYear BETWEEN 2000 AND 2024 
             and releasedMonth in ('Jan', 'Feb' ,'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec')
             GROUP BY releasedYear, releasedMonth
             ORDER BY releasedYear, month_count desc
         )
         
""")

app_year_month.show()
app_year_month.write.csv("output/releasedYear_bucket/app_year_month.csv", header=True)

+------------+-------------+-----------+----+
|releasedYear|releasedMonth|month_count|rank|
+------------+-------------+-----------+----+
|        2009|          Dec|         18|   1|
|        2009|          Jun|         14|   2|
|        2009|          May|         13|   3|
|        2009|          Jul|         10|   4|
|        2009|          Nov|          9|   5|
|        2009|          Feb|          8|   6|
|        2009|          Oct|          8|   6|
|        2009|          Sep|          7|   7|
|        2009|          Aug|          7|   7|
|        2009|          Mar|          6|   8|
|        2009|          Apr|          5|   9|
|        2010|          Dec|        859|   1|
|        2010|          Nov|        637|   2|
|        2010|          Oct|        588|   3|
|        2010|          Sep|        472|   4|
|        2010|          Aug|        471|   5|
|        2010|          Jul|        358|   6|
|        2010|          Jun|        356|   7|
|        2010|          May|      

In [188]:
#11) Most common month to release a app by developers
app_year_month.createOrReplaceTempView('app_year_month')
most_common_month = spark.sql(""" 
        SELECT releasedMonth, SUM(month_count) as total_count
        FROM app_year_month
        GROUP BY releasedMonth
"""
)
most_common_month.show()
most_common_month.write.csv("output/releasedYear_bucket/most_common_month.csv", header=True)

+-------------+-----------+
|releasedMonth|total_count|
+-------------+-----------+
|          Oct|     285517|
|          Sep|     301337|
|          Dec|     259551|
|          Aug|     301328|
|          May|     301357|
|          Jun|     315327|
|          Feb|     259107|
|          Nov|     249492|
|          Mar|     298285|
|          Jan|     268789|
|          Apr|     288517|
|          Jul|     316066|
+-------------+-----------+



In [186]:
# Operations on price bucket
genre = spark.table("genre_bucket")
genre_df = price.limit(5).toPandas()
genre_df

,Index,appId,developer,developerId,developerWebsite,free,genreId,inAppProductPrice,minInstalls,originalPrice,...,releasedDayYear,sale,score,title,releasedDay,releasedYear,releasedMonth,dateUpdated,minprice,maxprice
0,899795,app.sangreazul,3Dent medical s. r. o.,5142856662213353785,http://www.3Dent.sk,1,MEDICAL,None,100,None,...,"Jun 4, 2018",0,0.0,StrojCHECK by SangreAzul - Invisalign with A.I.,4,2018,Jun,2022-01-19 11:32:05,NaN,0.0
1,1000944,com.unitedstatesradios.radioquebec,Multiple Radios Online AM FM Apps,6263290699348208647,https://radiosamfmgratis.blogspot.com,1,MUSIC_AND_AUDIO,None,1,None,...,"Oct 31, 2020",0,0.0,US Radio Quebec App Radio List,31,2020,Oct,2020-10-31 19:15:07,NaN,0.0
2,899796,com.MobileApp.CareIs,CAREis,CAREis,http://www.careis.net,1,MEDICAL,None,100,None,...,"Jun 5, 2018",0,0.0,CAREis,5,2018,Jun,2022-06-21 14:49:55,NaN,0.0
3,1000945,com.audiostreamvolt.ecochiquiano,AudioStreamVolt.com,5057669035764163988,https://www.streamingeconomico.com/,1,MUSIC_AND_AUDIO,None,5,None,...,"Nov 29, 2020",0,0.0,EcoChiquiano,29,2020,Nov,2020-12-01 16:41:33,NaN,0.0
4,899797,com.sky.injurytrak,InjuryTrak Software,InjuryTrak+Software,http://injurytrak.com,1,MEDICAL,None,50,None,...,"Jun 5, 2018",0,0.0,InjuryTrak,5,2018,Jun,2020-08-25 16:21:43,NaN,0.0


In [193]:
#12) Which genre has most paid app
genre.createOrReplaceTempView('genre_table')
most_paid_genre = spark.sql(""" 
    SELECT genreId, COUNT(*) AS paid_app_count
    FROM genre_table
    WHERE  free = 0
    GROUP BY genreId
    ORDER BY paid_app_count DESC
"""
)
#13) Which genre has most free app
genre.createOrReplaceTempView('genre_table')
most_free_genre = spark.sql(""" 
    SELECT genreId, COUNT(*) AS free_app_count
    FROM genre_table
    WHERE  free = 1
    GROUP BY genreId
    ORDER BY free_app_count DESC
"""
)

most_paid_genre.show()
most_free_genre.show()
joined_result = most_paid_genre.join(most_free_genre, on="genreId", how="inner")
joined_result.show()
joined_result.write.csv("output/genreBucket_bucket/joined_result_free_paid.csv", header=True)

+-------------------+--------------+
|            genreId|paid_app_count|
+-------------------+--------------+
|    PERSONALIZATION|         19534|
|          EDUCATION|         15027|
|              TOOLS|          8809|
|BOOKS_AND_REFERENCE|          8671|
|      ENTERTAINMENT|          4403|
|       PRODUCTIVITY|          4009|
|    MUSIC_AND_AUDIO|          3146|
|        GAME_PUZZLE|          2994|
|          LIFESTYLE|          2741|
| HEALTH_AND_FITNESS|          2629|
|   TRAVEL_AND_LOCAL|          2509|
|   GAME_EDUCATIONAL|          2496|
|             SPORTS|          2415|
|            MEDICAL|          2280|
|        GAME_ARCADE|          2145|
|     GAME_ADVENTURE|          1946|
|        GAME_CASUAL|          1863|
|           BUSINESS|          1789|
|            FINANCE|          1491|
|        GAME_ACTION|          1477|
+-------------------+--------------+
only showing top 20 rows

+-------------------+--------------+
|            genreId|free_app_count|
+-----------

In [203]:
#14)How many genre contain in app purchase and also are paid app
most_paid_genre_inApp =spark.sql("""
    SELECT genreId, COUNT(*) AS paid_app_count
    FROM genre_table
    WHERE free = 0 and inAppProductPrice != 'None'
    GROUP BY genreId
    ORDER BY paid_app_count DESC
""")
most_paid_genre_inApp.show()


#15) How many genre does not contain in app purchase and also are free app

most_free_inApp =spark.sql("""
    SELECT genreId, COUNT(*) AS free_count
    FROM genre_table
    WHERE free = 1 and inAppProductPrice = 'None'
    GROUP BY genreId
    ORDER BY free_count DESC
""")
most_free_inApp.show()
joined_result = most_paid_genre_inApp.join(most_free_inApp, on="genreId", how="inner")
joined_result.show()
joined_result.write.csv("output/genreBucket_bucket/joined_result_purchase.csv", header=True)

+-------------------+--------------+
|            genreId|paid_app_count|
+-------------------+--------------+
|    PERSONALIZATION|          1461|
|MAPS_AND_NAVIGATION|           396|
|  GAME_ROLE_PLAYING|           215|
|        GAME_ACTION|           193|
|          EDUCATION|           183|
|        GAME_PUZZLE|           172|
|      GAME_STRATEGY|           170|
|              TOOLS|           159|
|    GAME_SIMULATION|           157|
|        GAME_ARCADE|           131|
|     GAME_ADVENTURE|           122|
|       PRODUCTIVITY|           118|
|      ENTERTAINMENT|           114|
|        GAME_CASUAL|           100|
|BOOKS_AND_REFERENCE|            99|
|             SPORTS|            96|
|         GAME_BOARD|            84|
| HEALTH_AND_FITNESS|            74|
|           BUSINESS|            65|
|          GAME_CARD|            56|
+-------------------+--------------+
only showing top 20 rows

+-------------------+----------+
|            genreId|free_count|
+-------------------